In [ ]:
# Data processing
# ==============================================================================
import numpy as np
import pandas as pd
from feature_engineering import codify_date_2, remove_outliers, covid_19_2
from feature_engineering import add_weather, one_hot_encode, cyclic_transform
from utils import handle_missing_values
from pathlib import Path

# Plots
# ==============================================================================
import matplotlib.pyplot as plt

# Modelling and Forecasting
# ==============================================================================
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.preprocessing import RollingFeatures
from skforecast.preprocessing import series_long_to_dict
from skforecast.preprocessing import exog_long_to_dict
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import HistGradientBoostingRegressor

/opt/homebrew/Caskroom/miniforge/base/envs/test_bike/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_parquet(Path("data") / "train.parquet")
data = codify_date_2(data)
data = remove_outliers(data)
data = covid_19_2(data)
data = add_weather(data)
data = handle_missing_values(data, "linear")
data = data.drop(columns=["counter_id", "site_id", "site_name", 
                                "counter_installation_date", 
                              "coordinates", "counter_technical_id",
                              "latitude", "longitude", "date", "bike_count"])
data.head()

/Users/joaosilva/Current/École Polytechnique/Python for Data Science/Project/bike_counters/feature_engineering.py:43: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  data.groupby(["counter_name", "date_truncated"])
/Users/joaosilva/Current/École Polytechnique/Python for Data Science/Project/bike_counters/feature_engineering.py:74: DtypeWarning: Columns (2,3,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  covid_19_index = pd.read_csv(Path("data") / "Covid_19_Index.csv")


Columns with missing values and their counts:
rr1         1326
ht_neige    7232
raf10       1650
etat_sol    8022
dtype: int64


,StringencyIndex_Average,counter_name,log_bike_count,datetime,year,month,day,day_of_week,hour,is_weekend,IsHoliday,t,rr1,u,ht_neige,raf10,ff,ww,etat_sol,tend
0,46.76,Face au 8 avenue de la porte de Charenton NO-SE,1.098612,2020-09-01 01:00:00,2020,9,1,1,1,False,False,285.75,0.0,81,0.0,2.4,1.6,1,0.0,-10
1,46.76,Voie Georges Pompidou NE-SO,0.000000,2020-09-01 01:00:00,2020,9,1,1,1,False,False,285.75,0.0,81,0.0,2.4,1.6,1,0.0,-10
2,46.76,20 Avenue de Clichy NO-SE,2.079442,2020-09-01 01:00:00,2020,9,1,1,1,False,False,285.75,0.0,81,0.0,2.4,1.6,1,0.0,-10
3,46.76,Pont des Invalides S-N,0.000000,2020-09-01 01:00:00,2020,9,1,1,1,False,False,285.75,0.0,81,0.0,2.4,1.6,1,0.0,-10
4,46.76,39 quai François Mauriac NO-SE,1.098612,2020-09-01 01:00:00,2020,9,1,1,1,False,False,285.75,0.0,81,0.0,2.4,1.6,1,0.0,-10


In [3]:
X_test = pd.read_parquet(Path("data") / "final_test.parquet")
X_test = codify_date_2(X_test)
X_test = covid_19_2(X_test)
X_test = add_weather(X_test)
X_test = handle_missing_values(X_test, "linear")

X_test.head()

/Users/joaosilva/Current/École Polytechnique/Python for Data Science/Project/bike_counters/feature_engineering.py:74: DtypeWarning: Columns (2,3,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  covid_19_index = pd.read_csv(Path("data") / "Covid_19_Index.csv")


Columns with missing values and their counts:
rr1         165
ht_neige    660
etat_sol    498
dtype: int64


,date,StringencyIndex_Average,counter_id,counter_name,site_id,site_name,counter_installation_date,coordinates,counter_technical_id,latitude,...,IsHoliday,t,rr1,u,ht_neige,raf10,ff,ww,etat_sol,tend
0,2021-09-10,43.77,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,False,291.45,0.0,94,0.0,3.4,2.0,0,1.0,0
1,2021-09-10,43.77,100036718-103036718,39 quai François Mauriac SE-NO,100036718,39 quai François Mauriac,2017-07-12,"48.83436,2.377",Y2H17021629,48.834360,...,False,291.45,0.0,94,0.0,3.4,2.0,0,1.0,0
2,2021-09-10,43.77,100057380-104057380,Totem Cours la Reine E-O,100057380,Totem Cours la Reine,2020-02-11,"48.86462,2.31444",YTH19111509,48.864620,...,False,291.45,0.0,94,0.0,3.4,2.0,0,1.0,0
3,2021-09-10,43.77,100056223-SC,Pont des Invalides N-S,100056223,Pont des Invalides N-S,2019-11-07,"48.86284,2.310345",Y2H19070365,48.862840,...,False,291.45,0.0,94,0.0,3.4,2.0,0,1.0,0
4,2021-09-10,43.77,100056226-104056226,Face au 8 avenue de la porte de Charenton NO-SE,100056226,Face au 8 avenue de la porte de Charenton,2019-11-01,"48.830331,2.400551",Y2H19070370,48.830331,...,False,291.45,0.0,94,0.0,3.4,2.0,0,1.0,0


In [4]:
series_train = data[["counter_name", "datetime", "log_bike_count"]]
exog_train = data[["counter_name", "datetime", 't', 'rr1', 'u', 'ht_neige', 'raf10', 'ff', 'ww', 'etat_sol', 'tend', "StringencyIndex_Average", "year", "month", "hour", "day_of_week", "IsHoliday"]]

categorical_columns = ["year", "month", "day", "hour", "day_of_week", "IsHoliday"]

# Apply transformations
exog_train["tend"] = exog_train["tend"].astype(float)
exog_train["u"] = exog_train["u"].astype(float)
exog_train["ww"] = exog_train["ww"].astype(float)
exog_train["year"] = exog_train["year"].astype(float)
exog_train["IsHoliday"] = exog_train["IsHoliday"].astype(float)
exog_train = cyclic_transform(exog_train, "hour", 24)
exog_train = one_hot_encode(exog_train, ["month", "day_of_week"])
exog_train.head()

/var/folders/b3/l_t9dzy13rz2smp7vyx9352h0000gn/T/ipykernel_22565/1195950012.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exog_train["tend"] = exog_train["tend"].astype(float)
/var/folders/b3/l_t9dzy13rz2smp7vyx9352h0000gn/T/ipykernel_22565/1195950012.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exog_train["u"] = exog_train["u"].astype(float)
/var/folders/b3/l_t9dzy13rz2smp7vyx9352h0000gn/T/ipykernel_22565/1195950012.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

,counter_name,datetime,t,rr1,u,ht_neige,raf10,ff,ww,etat_sol,...,month_10,month_11,month_12,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6
0,Face au 8 avenue de la porte de Charenton NO-SE,2020-09-01 01:00:00,285.75,0.0,81.0,0.0,2.4,1.6,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,Voie Georges Pompidou NE-SO,2020-09-01 01:00:00,285.75,0.0,81.0,0.0,2.4,1.6,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,20 Avenue de Clichy NO-SE,2020-09-01 01:00:00,285.75,0.0,81.0,0.0,2.4,1.6,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,Pont des Invalides S-N,2020-09-01 01:00:00,285.75,0.0,81.0,0.0,2.4,1.6,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,39 quai François Mauriac NO-SE,2020-09-01 01:00:00,285.75,0.0,81.0,0.0,2.4,1.6,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [33]:
series_dict_train = series_long_to_dict(
    data      = series_train,
    series_id = 'counter_name',
    index     = 'datetime',
    values    = 'log_bike_count',
    freq      = 'H'
)

exog_dict_train = exog_long_to_dict(
    data      = exog_train,
    series_id = 'counter_name',
    index     = 'datetime',
    freq      = 'H'
)

/opt/homebrew/Caskroom/miniforge/base/envs/test_bike/lib/python3.12/site-packages/skforecast/preprocessing/preprocessing.py:418: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  original_sizes = data.groupby(series_id).size()
/opt/homebrew/Caskroom/miniforge/base/envs/test_bike/lib/python3.12/site-packages/skforecast/preprocessing/preprocessing.py:420: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for k, v in data.groupby(series_id):
/opt/homebrew/Caskroom/miniforge/base/envs/test_bike/lib/python3.12/site-packages/skforecast/preprocessing/preprocessing.py:421: FutureWarning: 'H' is deprecated and will be removed

In [ ]:
exog_test = X_test[["counter_name", "datetime", 't', 'rr1', 'u', 'ht_neige', 'raf10', 'ff', 'ww', 'etat_sol', 'tend', "StringencyIndex_Average", "year", "month", "hour", "day_of_week", "IsHoliday"]]


# Apply sine and cosine transformations
exog_test["tend"] = exog_test["tend"].astype(float)
exog_test["u"] = exog_test["u"].astype(float)
exog_test["ww"] = exog_test["ww"].astype(float)
exog_test["year"] = exog_test["year"].astype(float)
exog_test["IsHoliday"] = exog_test["IsHoliday"].astype(float)
exog_test = cyclic_transform(exog_test, "hour", 24)
exog_test = one_hot_encode(exog_test, ["month", "day_of_week"])
exog_test.head()

/var/folders/b3/l_t9dzy13rz2smp7vyx9352h0000gn/T/ipykernel_19967/2996926702.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exog_test["tend"] = exog_test["tend"].astype(float)
/var/folders/b3/l_t9dzy13rz2smp7vyx9352h0000gn/T/ipykernel_19967/2996926702.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exog_test["u"] = exog_test["u"].astype(float)
/var/folders/b3/l_t9dzy13rz2smp7vyx9352h0000gn/T/ipykernel_19967/2996926702.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

,counter_name,datetime,t,rr1,u,ht_neige,raf10,ff,ww,etat_sol,...,hour_cos,month_9,month_10,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6
0,28 boulevard Diderot E-O,2021-09-10 01:00:00,291.45,0.0,94.0,0.0,3.4,2.0,0.0,1.0,...,0.965926,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,39 quai François Mauriac SE-NO,2021-09-10 01:00:00,291.45,0.0,94.0,0.0,3.4,2.0,0.0,1.0,...,0.965926,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,Totem Cours la Reine E-O,2021-09-10 01:00:00,291.45,0.0,94.0,0.0,3.4,2.0,0.0,1.0,...,0.965926,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,Pont des Invalides N-S,2021-09-10 01:00:00,291.45,0.0,94.0,0.0,3.4,2.0,0.0,1.0,...,0.965926,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,Face au 8 avenue de la porte de Charenton NO-SE,2021-09-10 01:00:00,291.45,0.0,94.0,0.0,3.4,2.0,0.0,1.0,...,0.965926,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [35]:
exog_dict_test = exog_long_to_dict(
    data      = exog_test,
    series_id = 'counter_name',
    index     = 'datetime',
    freq      = 'H'
)

/opt/homebrew/Caskroom/miniforge/base/envs/test_bike/lib/python3.12/site-packages/skforecast/preprocessing/preprocessing.py:478: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  original_sizes = data.groupby(series_id).size()
/opt/homebrew/Caskroom/miniforge/base/envs/test_bike/lib/python3.12/site-packages/skforecast/preprocessing/preprocessing.py:479: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  exog_dict = dict(tuple(data.groupby(series_id)))
/opt/homebrew/Caskroom/miniforge/base/envs/test_bike/lib/python3.12/site-packages/skforecast/preprocessing/preprocessing.py:481: FutureWarning: 'H' is deprecated and wil

In [36]:
# Fit forecaster
# ==============================================================================
window_features = RollingFeatures(stats=['mean', 'mean'], window_sizes=[24, 168])
forecaster = ForecasterRecursiveMultiSeries(
                regressor          = HistGradientBoostingRegressor(random_state=123),
                lags               = [1, 24, 168],
                window_features    = window_features,
                encoding           = "ordinal",
                dropna_from_series = False
            )

forecaster.fit(series=series_dict_train, exog=exog_dict_train, suppress_warnings=True)
forecaster

============================== 
ForecasterRecursiveMultiSeries 
============================== 
Regressor: HistGradientBoostingRegressor 
Lags: [  1  24 168] 
Window features: ['roll_mean_24', 'roll_mean_168'] 
Window size: 168 
Series encoding: ordinal 
Series names (levels): 
    152 boulevard du Montparnasse E-O, 152 boulevard du Montparnasse O-E, 18 quai de
    l'Hôtel de Ville NO-SE, 18 quai de l'Hôtel de Ville SE-NO, 20 Avenue de
    Clichy NO-SE, 20 Avenue de Clichy SE-NO, 254 rue de Vaugirard NE-SO, 254 rue
    de Vaugirard SO-NE, 27 quai de la Tournelle NO-SE, 27 quai de la Tournelle
    SE-NO, 28 boulevard Diderot E-O, 28 boulevard Diderot O-E, 36 quai de
    Grenelle NE-SO, 36 quai de Grenelle SO-NE, 38 rue Turbigo NE-SO, 38 rue
    Turbigo SO-NE, 39 quai François Mauriac NO-SE, 39 quai François Mauriac SE-
    NO, 6 rue Julia Bartet NE-SO, 6 rue Julia Bartet SO-NE, 67 boulevard
    Voltaire SE-NO, 90 Rue De Sèvres NE-SO, 90 Rue De Sèvres SO-NE, Face 104 rue
    d'Aubervilliers N-S, Face 104 rue d'Aubervilliers S-N, ..., Face au 48 quai
    de la marne NE-SO, Face au 48 quai de la marne SO-NE, Face au 70 quai de
    Bercy N-S, Face au 70 quai de Bercy S-N, Face au 8 avenue de la porte de
    Charenton NO-SE, Face au 8 avenue de la porte de Charenton SE-NO, Pont
    Charles De Gaulle NE-SO, Pont Charles De Gaulle SO-NE, Pont de Bercy NE-SO,
    Pont de Bercy SO-NE, Pont de la Concorde S-N, Pont des Invalides N-S, Pont
    des Invalides S-N, Quai d'Orsay E-O, Quai d'Orsay O-E, Totem 64 Rue de
    Rivoli E-O, Totem 64 Rue de Rivoli O-E, Totem 73 boulevard de Sébastopol
    N-S, Totem 73 boulevard de Sébastopol S-N, Totem 85 quai d'Austerlitz NO-SE,
    Totem 85 quai d'Austerlitz SE-NO, Totem Cours la Reine E-O, Totem Cours la
    Reine O-E, Voie Georges Pompidou NE-SO, Voie Georges Pompidou SO-NE 
Exogenous included: True 
Exogenous names: 
    t, rr1, u, ht_neige, raf10, ff, ww, etat_sol, tend, StringencyIndex_Average,
    year, IsHoliday, hour_sin, hour_cos, month_1, month_2, month_3, month_4,
    month_5, month_6, month_7, month_8, month_9, month_10, month_11, month_12,
    day_of_week_0, day_of_week_1, day_of_week_2, day_of_week_3, day_of_week_4,
    day_of_week_5, day_of_week_6 
Transformer for series: None 
Transformer for exog: None 
Weight function included: False 
Series weights: None 
Differentiation order: None 
Training range: 
    '152 boulevard du Montparnasse E-O': ['2020-09-01 01:00:00', '2021-09-09
    23:00:00'], '152 boulevard du Montparnasse O-E': ['2020-09-01 01:00:00',
    '2021-09-09 23:00:00'], '18 quai de l'Hôtel de Ville NO-SE': ['2020-09-01
    01:00:00', '2021-09-09 23:00:00'], '18 quai de l'Hôtel de Ville SE-NO':
    ['2020-09-01 01:00:00', '2021-09-09 23:00:00'], '20 Avenue de Clichy NO-SE':
    ['2020-09-01 01:00:00', '2021-09-09 23:00:00'], ..., 'Totem 85 quai
    d'Austerlitz SE-NO': ['2020-09-01 01:00:00', '2021-09-09 23:00:00'], 'Totem
    Cours la Reine E-O': ['2020-09-01 01:00:00', '2021-09-09 23:00:00'], 'Totem
    Cours la Reine O-E': ['2020-09-01 01:00:00', '2021-09-09 23:00:00'], 'Voie
    Georges Pompidou NE-SO': ['2020-09-01 01:00:00', '2021-09-09 23:00:00'],
    'Voie Georges Pompidou SO-NE': ['2020-09-01 01:00:00', '2021-09-09
    23:00:00'] 
Training index type: DatetimeIndex 
Training index frequency: h 
Regressor parameters: 
    {'categorical_features': 'warn', 'early_stopping': 'auto', 'interaction_cst':
    None, 'l2_regularization': 0.0, 'learning_rate': 0.1, 'loss':
    'squared_error', 'max_bins': 255, 'max_depth': None, 'max_features': 1.0,
    'max_iter': 100, 'max_leaf_nodes': 31, 'min_samples_leaf': 20,
    'monotonic_cst': None, 'n_iter_no_change': 10, 'quantile': None,
    'random_state': 123, 'scoring': 'loss', 'tol': 1e-07, 'validation_fraction':
    0.1, 'verbose': 0, 'warm_start': False} 
fit_kwargs: {} 
Creation date: 2024-12-11 19:53:24 
Last fit date: 2024-12-11 19:53:37 
Skforecast version: 0.14.0 
Python version: 3.12.4 
Forecaster id: None

In [37]:
predictions = forecaster.predict(steps=1020, exog=exog_dict_test)

/opt/homebrew/Caskroom/miniforge/base/envs/test_bike/lib/python3.12/site-packages/skforecast/utils/utils.py:966: MissingValuesWarning: `exog` for series '152 boulevard du Montparnasse E-O' doesn't have as many values as steps predicted, 1020. Missing values are filled with NaN. Most of machine learning models do not allow missing values. Prediction method may fail. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=MissingValuesWarning)
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/test_bike/lib/python3.12/site-packages/skforecast/utils/utils.py:984: MissingExogWarning: {'month_12', 'month_1', 'month_8', 'month_4', 'month_7', 'month_11', 'month_5', 'month_6', 'month_3', 'month_2'} not present in `exog` for series '152 boulevard du Montparnasse E-O'. All values will be NaN. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=MissingExogWarning)
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/test_bike/lib

In [38]:
predictions = predictions.reset_index(names='datetime').melt(id_vars=['datetime'], var_name='counter_name', value_name='log_bike_count')
predictions.head()

,datetime,counter_name,log_bike_count
0,2021-09-10 00:00:00,152 boulevard du Montparnasse E-O,1.921563
1,2021-09-10 01:00:00,152 boulevard du Montparnasse E-O,1.236673
2,2021-09-10 02:00:00,152 boulevard du Montparnasse E-O,0.946127
3,2021-09-10 03:00:00,152 boulevard du Montparnasse E-O,1.133086
4,2021-09-10 04:00:00,152 boulevard du Montparnasse E-O,2.581472


In [41]:
X_original = pd.read_parquet(Path("data") / "final_test.parquet")
X_original.loc[:, "index1"] = X_original.index
X_original = codify_date_2(X_original)
X_original.head()

,counter_id,counter_name,site_id,site_name,date,counter_installation_date,coordinates,counter_technical_id,latitude,longitude,index1,datetime,year,month,day,day_of_week,hour,is_weekend,IsHoliday
0,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2021-09-10,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0,2021-09-10 01:00:00,2021,9,10,4,1,False,False
1,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2021-09-10,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,1,2021-09-10 13:00:00,2021,9,10,4,13,False,False
2,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2021-09-10,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,2,2021-09-10 17:00:00,2021,9,10,4,17,False,False
3,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2021-09-10,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,3,2021-09-10 19:00:00,2021,9,10,4,19,False,False
4,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2021-09-10,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,4,2021-09-10 22:00:00,2021,9,10,4,22,False,False


In [42]:
merged_df = X_original.merge(predictions, on=['counter_name', 'datetime'], how='left')
merged_df.head()

,counter_id,counter_name,site_id,site_name,date,counter_installation_date,coordinates,counter_technical_id,latitude,longitude,index1,datetime,year,month,day,day_of_week,hour,is_weekend,IsHoliday,log_bike_count
0,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2021-09-10,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0,2021-09-10 01:00:00,2021,9,10,4,1,False,False,0.190413
1,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2021-09-10,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,1,2021-09-10 13:00:00,2021,9,10,4,13,False,False,1.823540
2,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2021-09-10,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,2,2021-09-10 17:00:00,2021,9,10,4,17,False,False,1.847571
3,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2021-09-10,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,3,2021-09-10 19:00:00,2021,9,10,4,19,False,False,0.830286
4,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2021-09-10,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,4,2021-09-10 22:00:00,2021,9,10,4,22,False,False,0.395662


In [44]:
results = pd.DataFrame(
    dict(
        Id=np.arange(len(merged_df["log_bike_count"])),
        log_bike_count=merged_df["log_bike_count"],
    )
)
results.to_csv("submission.csv", index=False)

In [5]:
import session_info
session_info.show(html=False)

-----
feature_engineering NA
matplotlib          3.9.2
numpy               2.0.2
pandas              2.2.3
session_info        1.0.0
skforecast          0.14.0
sklearn             1.5.2
utils               NA
-----
IPython             8.30.0
jupyter_client      8.6.3
jupyter_core        5.7.2
jupyterlab          4.3.3
notebook            7.3.1
-----
Python 3.12.4 | packaged by conda-forge | (main, Jun 17 2024, 10:13:44) [Clang 16.0.6 ]
macOS-15.0.1-arm64-arm-64bit
-----
Session information updated at 2024-12-11 20:37


In [ ]:
pip install matplotlib==3.9.2
pip install numpy==2.0.2
pip install pandas==2.2.3
pip install scikit-learn==1.5.2
pip install skforecast==0.14.0

SyntaxError: invalid syntax (209920656.py, line 1)